# Stokes equations

This demo is implemented in a single Python file,
demo\_stokes-iterative.py, which contains both the variational forms and
the solver.

## Implementation

This description goes through the implementation (in
demo\_stokes-iterative.py) of a solver for the above described Stokes
equations. Some of the standard steps will be described in less detail,
so before reading this, we suggest that you are familiarize with the
Poisson demo
&lt;demo\_pde\_poisson\_python\_documentation&gt; (for the very basics)
and the Mixed Poisson demo
&lt;demo\_pde\_mixed-poisson\_python\_documentation&gt; (for how to deal
with mixed function spaces). Also, the Navier--Stokes demo
&lt;demo\_pde\_navier\_stokes\_python\_documentation&gt; illustrates how
to use iterative solvers in a more implicit manner (typically only
suitable for positive-definite systems of equations).

The Stokes equations as formulated above result in a system of linear
equations that is not positive-definite. Standard iterative linear
solvers typically fail to converge for such systems. Some care must
therefore be taken in preconditioning the systems of equations.
Moreover, not all of the linear algebra backends support this. We
therefore start by checking that either "PETSc" or "Tpetra" (from
Trilinos) is available. We also try to pick MINRES Krylov subspace
method which is suitable for symmetric indefinite problems. If not
available, costly QMR method is choosen.

In [ ]:
from dolfin import *

# Test for PETSc or Tpetra
if not has_linear_algebra_backend("PETSc") and not has_linear_algebra_backend("Tpetra"):
    info("DOLFIN has not been configured with Trilinos or PETSc. Exiting.")
    exit()

if not has_krylov_solver_preconditioner("amg"):
    info("Sorry, this demo is only available when DOLFIN is compiled with AMG "
         "preconditioner, Hypre or ML.")
    exit()

if has_krylov_solver_method("minres"):
    krylov_method = "minres"
elif has_krylov_solver_method("tfqmr"):
    krylov_method = "tfqmr"
else:
    info("Default linear algebra backend was not compiled with MINRES or TFQMR "
         "Krylov subspace method. Terminating.")
    exit()

Next, we define the mesh (a :pyUnitCubeMesh
&lt;dolfin.cpp.UnitCubeMesh&gt;) and a :pyMixedFunctionSpace
&lt;dolfin.functions.functionspace.MixedFunctionSpace&gt; composed of a
:pyVectorFunctionSpace
&lt;dolfin.functions.functionspace.VectorFunctionSpace&gt; of continuous
piecewise quadratics and a :pyFunctionSpace
&lt;dolfin.functions.functionspace.FunctionSpace&gt; of continuous
piecewise linears. (This mixed finite element space is known as the
Taylor--Hood elements and is a stable, standard element pair for the
Stokes equations.)

In [ ]:
# Load mesh
mesh = UnitCubeMesh(16, 16, 16)

# Define function spaces
V = VectorFunctionSpace(mesh, "CG", 2)
Q = FunctionSpace(mesh, "CG", 1)
W = V * Q

Next, we define the boundary conditions.

In [ ]:
# Boundaries
def right(x, on_boundary): return x[0] > (1.0 - DOLFIN_EPS)
def left(x, on_boundary): return x[0] < DOLFIN_EPS
def top_bottom(x, on_boundary):
    return x[1] > 1.0 - DOLFIN_EPS or x[1] < DOLFIN_EPS

# No-slip boundary condition for velocity
noslip = Constant((0.0, 0.0, 0.0))
bc0 = DirichletBC(W.sub(0), noslip, top_bottom)

# Inflow boundary condition for velocity
inflow = Expression(("-sin(x[1]*pi)", "0.0", "0.0"))
bc1 = DirichletBC(W.sub(0), inflow, right)

# Collect boundary conditions
bcs = [bc0, bc1]

The bilinear and linear forms corresponding to the weak mixed
formulation of the Stokes equations are defined as follows:

In [ ]:
# Define variational problem
(u, p) = TrialFunctions(W)
(v, q) = TestFunctions(W)
f = Constant((0.0, 0.0, 0.0))
a = inner(grad(u), grad(v))*dx + div(v)*p*dx + q*div(u)*dx
L = inner(f, v)*dx

We can now use the same :pyTrialFunctions
&lt;dolfin.functions.function.TrialFunction&gt; and
:pyTestFunctions &lt;dolfin.functions.function.TestFunction&gt; to
define the preconditioner matrix. We first define the form corresponding
to the expression for the preconditioner (given in the initial
description above):

In [ ]:
# Form for use in constructing preconditioner matrix
b = inner(grad(u), grad(v))*dx + p*q*dx

Next, we want to assemble the matrix corresponding to the bilinear form
and the vector corresponding to the linear form of the Stokes equations.
Moreover, we want to apply the specified boundary conditions to the
linear system. However, :pyassembling
&lt;dolfin.fem.assembling.assemble&gt; the matrix and vector and
applying a :pyDirichletBC &lt;dolfin.fem.bcs.DirichletBC&gt; separately
will possibly result in a non-symmetric system of equations. Instead, we
can use the :pyassemble\_system
&lt;dolfin.fem.assembling.assemble\_system&gt; function to assemble both
the matrix `A`, the vector `bb`, and apply the boundary conditions `bcs`
in a symmetric fashion:

In [ ]:
# Assemble system
A, bb = assemble_system(a, L, bcs)

We do the same for the preconditioner matrix `P` using the linear form
`L` as a dummy form:

In [ ]:
# Assemble preconditioner system
P, btmp = assemble_system(b, L, bcs)

Next, we specify the iterative solver we want to use, in this case a
:pyKrylovSolver &lt;dolfin.cpp.KrylovSolver&gt;. We associate the
left-hand side matrix `A` and the preconditioner matrix `P` with the
solver by calling :pysolver.set\_operators
&lt;dolfin.cpp.GenericLinearSolver.set\_operators&gt;.

In [ ]:
# Create Krylov solver and AMG preconditioner
solver = KrylovSolver(krylov_method, "amg")

# Associate operator (A) and preconditioner matrix (P)
solver.set_operators(A, P)

We are now almost ready to solve the linear system of equations. It
remains to specify a :pyVector &lt;dolfin.cpp.Vector&gt; for storing the
result. For easy manipulation later, we can define a
:pyFunction &lt;dolfin.functions.function.Function&gt; and use the
vector associated with this Function. The call to
:pysolver.solve &lt;dolfin.cpp.KrylovSolver.solve&gt; then looks as
follows

In [ ]:
# Solve
U = Function(W)
solver.solve(U.vector(), bb)

Finally, we can play with the result in different ways:

In [ ]:
# Get sub-functions
u, p = U.split()

# Save solution in VTK format
ufile_pvd = File("velocity.pvd")
ufile_pvd << u
pfile_pvd = File("pressure.pvd")
pfile_pvd << p

# Plot solution
plot(u)
plot(p)
interactive()

## Complete code